In [23]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [24]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["score", "cards", "aces"]

In [25]:
GAME_COLUMNS = ["chance"] + ["player_"+colname for colname in HAND_COLUMNS] + ["dealer_"+colname for colname in HAND_COLUMNS] + DECK_COLUMNS

In [26]:
def get_deck(number_decks=6):
    """get a deck with the specified number of decks (default: 6)"""
    
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [27]:
def get_game(deck=None, player_cards=[], dealer_cards=[], update_deck=True):
    """get a game with the specified deck, player cards and dealer cards"""

    if deck is None:
        deck = get_deck()
    
    game = pd.DataFrame(
        [1.0] + [0, 0, 0]*2 + deck.values.flatten().tolist(), 
        index=GAME_COLUMNS
    ).T.astype(int).astype({"chance":float})

    for player_card_value in player_cards:
        game = update_player_with_card(game, player_card_value, probabilistic=False, update_deck=update_deck)
    
    for dealer_card_value in dealer_cards:
        game = update_dealer_with_card(game, dealer_card_value, probabilistic=False, update_deck=update_deck)
    
    return game

In [28]:
def update_player_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the player drawing a card with the specified value"""

    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["player_score"] += value
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game["player_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    return next_game

In [29]:
def update_dealer_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the dealer drawing a card with the specified value"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["dealer_score"] += value
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game["dealer_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -1
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1 

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    return next_game

In [30]:
def player_hits_once(game):
    """get updated game when player hits once, throws error for finished hands"""

    assert np.all(game["dealer_cards"] != 0), f"dealer must have one card showing. hands: {game}"    
    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to hit. hands:\n{game}"
    assert game[DECK_COLUMNS].sum(axis=1).max() == game[DECK_COLUMNS].sum(axis=1).min(), f"there are decks with different number of cards. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["player_score"] += next_game["drawn"]
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "player_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    # removing drawn column
    next_game = next_game.drop("drawn", axis="columns")
    
    return next_game.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()[GAME_COLUMNS].sort_values("player_score")


In [31]:
def player_hits_once_where_possible(game):
    """get updated game when player hits once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  

    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)
    
    return final.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()[GAME_COLUMNS].sort_values("player_score")

In [43]:
def dealer_draws_once(game):
    """get updated game when dealer draws one card, throws error for finished hands"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to draw. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["dealer_score"] += next_game["drawn"]
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_score"] == 2), "dealer_score"] = 22

    # removing drawn column
    next_game = next_game.drop("drawn", axis="columns")
    
    return next_game.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()[GAME_COLUMNS].sort_values("dealer_score")

In [44]:
def dealer_plays(game):
    """get updated game when dealer plays until 17 or bust"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    if len(playable) == 0:
        print("Dealer has finished all hands")
        return game

    while len(playable) > 0:
        next_possible = dealer_draws_once(playable)

        is_playable = (next_possible["dealer_score"] < 17) & (next_possible["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_possible[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_possible[is_playable]
    
    return final

In [45]:
def get_return_chances_finished(game):
    """get the probabilities of each possible return of the current state for the specified game
    no further moves are made"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    # blackjack win
    blackjack_win = (game["player_score"] == 22) & (game["dealer_score"] != 22)
    blackjack_win_chance = game.loc[blackjack_win, "chance"].sum()

    # win
    win = (game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22)
    win_chance = game.loc[win, "chance"].sum()

    # lose
    lose = (game["player_score"] < game["dealer_score"])
    lose_chance = game.loc[lose, "chance"].sum()

    # push
    push = (game["player_score"] == game["dealer_score"])
    push_chance = game.loc[push, "chance"].sum()

    expected_return = blackjack_win_chance*1.5 + win_chance*1 + push_chance*0 + lose_chance*(-1)

    return pd.Series({
        "blackjack_win_chance": blackjack_win_chance,
        "win_chance": win_chance,
        "push_chance": push_chance,
        "lose_chance": lose_chance,
        "expected_return": expected_return
    })

In [46]:
def get_return_chances_standing(game, verbose=False):
    """get the expected return of standing (or doing nothing in case the player is already finished)"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = dealer_plays(game.copy())

    if verbose : print(game)

    return get_return_chances_finished(game)

In [47]:
def get_return_chances_hitting_once(game, verbose=False):
    """get the expected return of hitting exactly once, throws error for finished hands"""
    
    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    return get_return_chances_standing(game, verbose=verbose)

In [48]:
def get_return_chances_hitting_once_where_possible(game, verbose=False):
    """get the expected return of hitting exactly once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    game = player_hits_once_where_possible(game.copy())

    return get_return_chances_standing(game, verbose=verbose)

In [49]:
def get_expected_return_doubling_down(game, verbose=False):
    """get the expected return of doubling down, throws error for finished hands"""

    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] == 2), "player does not have exactly 2 cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    return_chances = get_return_chances_standing(game, verbose=verbose)
    return_chances["expected_return"] *= 2
    
    return return_chances

In [50]:
def get_expected_return_doubling_down_where_possible(game, verbose=False):
    """get the expected return of doubling down, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)

    return_chances = get_return_chances_standing(final, verbose=verbose)
    return_chances["expected_return"] *= 2
    
    return return_chances

In [51]:
game = get_game(None, [5,4], [6])

print(pd.DataFrame({
    "standing":get_return_chances_standing(game, True), 
    "hitting_once":get_return_chances_standing(player_hits_once_where_possible(game), True), 
    "hitting_twice":get_return_chances_standing(player_hits_once_where_possible(player_hits_once_where_possible(game)), True), 
    "hitting_thrice":get_return_chances_standing(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(game))), True),
    "hitting_four_times":get_return_chances_standing(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(game)))), True),
    "doubling":get_expected_return_doubling_down(game, True)
    })
)

           chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0    7.766990e-02             9             2            0            17             2            1  24  24  23  23  23  24  24  24  96  23
1    4.640020e-02             9             2            0            -1             3            0  24  24  23  23  22  24  24  24  95  24
2    1.210440e-02             9             2            0            -1             3            0  24  24  23  23  23  23  24  23  96  24
3    4.841760e-02             9             2            0            -1             3            0  24  24  23  23  23  23  24  24  95  24
4    5.800025e-03             9             2            0            -1             3            0  24  24  23  23  23  24  22  24  96  24
..            ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
674  1.781556e-09   

In [ ]:
game = get_game()

In [ ]:
game = dealer_draws_once(game)
game

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,0.076923,0,0,0,2,1,0,23,24,24,24,24,24,24,24,96,24
1,0.076923,0,0,0,3,1,0,24,23,24,24,24,24,24,24,96,24
2,0.076923,0,0,0,4,1,0,24,24,23,24,24,24,24,24,96,24
3,0.076923,0,0,0,5,1,0,24,24,24,23,24,24,24,24,96,24
4,0.076923,0,0,0,6,1,0,24,24,24,24,23,24,24,24,96,24
5,0.076923,0,0,0,7,1,0,24,24,24,24,24,23,24,24,96,24
6,0.076923,0,0,0,8,1,0,24,24,24,24,24,24,23,24,96,24
7,0.076923,0,0,0,9,1,0,24,24,24,24,24,24,24,23,96,24
8,0.307692,0,0,0,10,1,0,24,24,24,24,24,24,24,24,95,24
9,0.076923,0,0,0,11,1,1,24,24,24,24,24,24,24,24,96,23


In [ ]:
game = player_hits_once_where_possible(game)
game = player_hits_once_where_possible(game)
game

Only 10 games are playable
Only 100 games are playable


,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,0.000404,4,2,0,2,1,0,21,24,24,24,24,24,24,24,96,24
1,0.000440,4,2,0,3,1,0,22,23,24,24,24,24,24,24,96,24
2,0.000440,4,2,0,4,1,0,22,24,23,24,24,24,24,24,96,24
3,0.000440,4,2,0,5,1,0,22,24,24,23,24,24,24,24,96,24
4,0.000440,4,2,0,6,1,0,22,24,24,24,23,24,24,24,96,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,0.003677,22,2,1,4,1,0,24,24,23,24,24,24,24,24,95,23
541,0.003677,22,2,1,3,1,0,24,23,24,24,24,24,24,24,95,23
548,0.014553,22,2,1,10,1,0,24,24,24,24,24,24,24,24,94,23
543,0.003677,22,2,1,5,1,0,24,24,24,23,24,24,24,24,95,23


In [ ]:
scores = get_return_chances_standing(game, True)
scores

              chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0       1.427808e-04            17             2            0            17             2            1  24  24  24  24  23  24  23  23  96  23
1       5.711232e-04            12             2            0            17             2            0  24  23  24  24  24  23  24  23  95  24
2       1.368316e-04            12             2            0            17             2            0  24  23  24  24  24  24  23  22  96  24
3       1.427808e-04             9             2            0            17             2            0  24  23  24  24  23  24  23  23  96  24
4       5.711232e-04             9             2            0            17             2            0  24  24  23  23  24  23  24  24  95  24
...              ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..

blackjack_win_chance    0.045323
win_chance              0.339357
push_chance             0.048233
lose_chance             0.567088
expected_return        -0.159747
dtype: float64

In [ ]:
scores = get_return_chances_hitting_once_where_possible(game, True)
scores

Only 540 games are playable
              chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0       6.397321e-05            19             3            0            17             2            0  24  24  24  22  24  23  24  23  95  24
1       1.532692e-05            19             3            0            17             2            0  24  24  24  22  24  24  23  22  96  24
2       1.335093e-04            19             3            0            17             2            0  24  24  24  23  23  23  23  24  95  24
3       3.198661e-05            19             3            0            17             2            0  24  24  24  23  23  24  22  23  96  24
4       5.864211e-05            19             3            0            17             2            0  24  24  24  23  24  21  24  24  95  24
...              ...           ...           ...          ...           ...           ...          ...  ..  ..  ..

blackjack_win_chance    0.045323
win_chance              0.264836
push_chance             0.047609
lose_chance             0.642231
expected_return        -0.309411
dtype: float64

In [ ]:
deck = get_deck()
player = get_hand(deck, [10,6])
dealer = get_hand(deck, [7])

r_stand = get_expected_return_standing(deck, player, dealer)
r_hit = get_expected_return_hitting_once(deck, player, dealer)

print("")
print("Expected return of standing is", r_stand)
print("Expected return of hitting once is", r_hit)

NameError: name 'get_hand' is not defined

In [ ]:
#TODO: implement chained card probabilities (player draws before dealer, so dealer's probabilities should be conditional on player's card)
#TODO: implement splitting
#TODO: implement doubling down